<a href="https://colab.research.google.com/github/ThienNguyen3001/Graph-Theory-ArangoDB-Final/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Chuẩn bị

* Gọi môi trường

In [1]:
%%capture
!git clone https://github.com/ThienNguyen3001/Graph-Theory-ArangoDB-Final.git
!rsync -av Graph-Theory-ArangoDB-Final/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [2]:
import json
import requests
import sys
import time

import oasis
from pyArango.connection import *
from arango import ArangoClient

* **Tạo cơ sở dữ liệu tạm thời**


In [3]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName='test',
                                 credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')

# Connect to the temp database
conn = oasis.connect(login)
db = conn[login['dbName']]

Requesting new temp credentials.
Temp database ready to use.


* **Xem thông tin DB đã tạo**


In [4]:
print('https://{}:{}'.format(login['hostname'], login['port']))
print('Username: ' + login['username'])
print('Password: ' + login['password'])
print('Database: ' + login['dbName'])


https://tutorials.arangodb.cloud:8529
Username: TUTfvh2z3gsmzu7z8igmk5egh
Password: TUT4z24ti87547un4x8kbh5sf
Database: TUT2qempb1xekda6pmct46qm


In [5]:
print(type(login), login, sep='\n')

<class 'dict'>
{'dbName': 'TUT2qempb1xekda6pmct46qm', 'username': 'TUTfvh2z3gsmzu7z8igmk5egh', 'password': 'TUT4z24ti87547un4x8kbh5sf', 'hostname': 'tutorials.arangodb.cloud', 'port': 8529}


#2 Tạo dữ liệu AQL

* **Tạo collection "Collection" trong DB**

Các bộ sưu tập trong ArangoDB. Chúng chỉ được thao tác thông qua các ngôn ngữ Ngôn ngữ truy vấn Arango NoSQL (AQL). Các hướng dẫn bên dưới tạo ra một bộ sưu tập và có thể liên quan đến các hướng dẫn DDL của ngôn ngữ SQL. Trong ví dụ này, nhóm sẽ tạo các nhân vật trong loạt phim "*Game of Thrones*".

In [6]:
collecName = 'Characters'
db.createCollection(name=collecName)

ArangoDB collection name: Characters, id: 658751834, type: document, status: loaded

* **Thêm data bằng Insert**

In [7]:
insert_query = """
INSERT {
    "name": "Ned",
    "surname": "Stark",
    "alive": true,
    "age": 41,
    "traits": ["A","H","C","N","P"]
} INTO Characters
"""

db.AQLQuery(insert_query)

* Lấy tất cả các nhân vật trong bản ghi

In [8]:
all_characters = """
FOR c IN Characters
    RETURN c
"""

query_result = db.AQLQuery(all_characters, rawResults=True)

for doc in  query_result:
    print(doc)
    print()

{'_key': '266274254873', '_id': 'Characters/266274254873', '_rev': '_i3-9TdC---', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'traits': ['A', 'H', 'C', 'N', 'P']}



* **Chèn các bản ghi**

In [9]:
insert_query = """
LET data = [
    { "name": "Robert", "surname": "Baratheon", "alive": false, "traits": ["A","H","C"] },
    { "name": "Jaime", "surname": "Lannister", "alive": true, "age": 36, "traits": ["A","F","B"] },
    { "name": "Catelyn", "surname": "Stark", "alive": false, "age": 40, "traits": ["D","H","C"] },
    { "name": "Cersei", "surname": "Lannister", "alive": true, "age": 36, "traits": ["H","E","F"] },
    { "name": "Daenerys", "surname": "Targaryen", "alive": true, "age": 16, "traits": ["D","H","C"] },
    { "name": "Jorah", "surname": "Mormont", "alive": false, "traits": ["A","B","C","F"] },
    { "name": "Petyr", "surname": "Baelish", "alive": false, "traits": ["E","G","F"] },
    { "name": "Viserys", "surname": "Targaryen", "alive": false, "traits": ["O","L","N"] },
    { "name": "Jon", "surname": "Snow", "alive": true, "age": 16, "traits": ["A","B","C","F"] },
    { "name": "Sansa", "surname": "Stark", "alive": true, "age": 13, "traits": ["D","I","J"] },
    { "name": "Arya", "surname": "Stark", "alive": true, "age": 11, "traits": ["C","K","L"] },
    { "name": "Robb", "surname": "Stark", "alive": false, "traits": ["A","B","C","K"] },
    { "name": "Theon", "surname": "Greyjoy", "alive": true, "age": 16, "traits": ["E","R","K"] },
    { "name": "Bran", "surname": "Stark", "alive": true, "age": 10, "traits": ["L","J"] },
    { "name": "Joffrey", "surname": "Baratheon", "alive": false, "age": 19, "traits": ["I","L","O"] },
    { "name": "Sandor", "surname": "Clegane", "alive": true, "traits": ["A","P","K","F"] },
    { "name": "Tyrion", "surname": "Lannister", "alive": true, "age": 32, "traits": ["F","K","M","N"] },
    { "name": "Khal", "surname": "Drogo", "alive": false, "traits": ["A","C","O","P"] },
    { "name": "Tywin", "surname": "Lannister", "alive": false, "traits": ["O","M","H","F"] },
    { "name": "Davos", "surname": "Seaworth", "alive": true, "age": 49, "traits": ["C","K","P","F"] },
    { "name": "Samwell", "surname": "Tarly", "alive": true, "age": 17, "traits": ["C","L","I"] },
    { "name": "Stannis", "surname": "Baratheon", "alive": false, "traits": ["H","O","P","M"] },
    { "name": "Melisandre", "alive": true, "traits": ["G","E","H"] },
    { "name": "Margaery", "surname": "Tyrell", "alive": false, "traits": ["M","D","B"] },
    { "name": "Jeor", "surname": "Mormont", "alive": false, "traits": ["C","H","M","P"] },
    { "name": "Bronn", "alive": true, "traits": ["K","E","C"] },
    { "name": "Varys", "alive": true, "traits": ["M","F","N","E"] },
    { "name": "Shae", "alive": false, "traits": ["M","D","G"] },
    { "name": "Talisa", "surname": "Maegyr", "alive": false, "traits": ["D","C","B"] },
    { "name": "Gendry", "alive": false, "traits": ["K","C","A"] },
    { "name": "Ygritte", "alive": false, "traits": ["A","P","K"] },
    { "name": "Tormund", "surname": "Giantsbane", "alive": true, "traits": ["C","P","A","I"] },
    { "name": "Gilly", "alive": true, "traits": ["L","J"] },
    { "name": "Brienne", "surname": "Tarth", "alive": true, "age": 32, "traits": ["P","C","A","K"] },
    { "name": "Ramsay", "surname": "Bolton", "alive": true, "traits": ["E","O","G","A"] },
    { "name": "Ellaria", "surname": "Sand", "alive": true, "traits": ["P","O","A","E"] },
    { "name": "Daario", "surname": "Naharis", "alive": true, "traits": ["K","P","A"] },
    { "name": "Missandei", "alive": true, "traits": ["D","L","C","M"] },
    { "name": "Tommen", "surname": "Baratheon", "alive": true, "traits": ["I","L","B"] },
    { "name": "Jaqen", "surname": "H'ghar", "alive": true, "traits": ["H","F","K"] },
    { "name": "Roose", "surname": "Bolton", "alive": true, "traits": ["H","E","F","A"] },
    { "name": "The High Sparrow", "alive": true, "traits": ["H","M","F","O"] }
]

FOR d IN data
    INSERT d INTO Characters
"""

db.AQLQuery(insert_query)

* Truy vấn cơ bản

In [10]:
all_characters_names = """
FOR name IN Characters
    RETURN name.name
"""

query_result = db.AQLQuery(all_characters_names, rawResults=True)

# Lấy ra chỉ 10 tên
for i in range(10):
  print(query_result[i])

Ned
Robert
Jaime
Catelyn
Cersei
Daenerys
Jorah
Petyr
Viserys
Jon


In [11]:
list(query_result)

['Ned',
 'Robert',
 'Jaime',
 'Catelyn',
 'Cersei',
 'Daenerys',
 'Jorah',
 'Petyr',
 'Viserys',
 'Jon',
 'Sansa',
 'Arya',
 'Robb',
 'Theon',
 'Bran',
 'Joffrey',
 'Sandor',
 'Tyrion',
 'Khal',
 'Tywin',
 'Davos',
 'Samwell',
 'Stannis',
 'Melisandre',
 'Margaery',
 'Jeor',
 'Bronn',
 'Varys',
 'Shae',
 'Talisa',
 'Gendry',
 'Ygritte',
 'Tormund',
 'Gilly',
 'Brienne',
 'Ramsay',
 'Ellaria',
 'Daario',
 'Missandei',
 'Tommen',
 'Jaqen',
 'Roose',
 'The High Sparrow']

* **Truy vấn có điều kiện**


In [12]:
find_ned_query = """
FOR c IN Characters
    FILTER c.alive == false
    RETURN c.name
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)
for doc in  query_result:
    print(doc, sep='\n')

Robert
Catelyn
Jorah
Petyr
Viserys
Robb
Joffrey
Khal
Tywin
Stannis
Margaery
Jeor
Shae
Talisa
Gendry
Ygritte


In [13]:
find_ned_query = """
FOR c IN Characters
    FILTER c.alive == true AND c.age >= 18
    RETURN c.name
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)

for doc in  query_result:
    print(doc, sep='\n')

Ned
Jaime
Cersei
Tyrion
Davos
Brienne


In [14]:
find_ned_query = """
FOR c IN Characters
    FILTER c.alive == false AND c.age >= 18
    RETURN { Name: c.name, Surname: c.surname, Age: c.age }
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)

for doc in  query_result:
    print(doc, sep='\n')

{'Name': 'Catelyn', 'Surname': 'Stark', 'Age': 40}
{'Name': 'Joffrey', 'Surname': 'Baratheon', 'Age': 19}


In [15]:
find_ned_query = """
FOR c IN Characters
    FILTER c.surname == "Bolton"
    RETURN {Name: c.name, Surname: c.surname}
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)
for doc in  query_result:
    print(doc, sep='\n')

{'Name': 'Ramsay', 'Surname': 'Bolton'}
{'Name': 'Roose', 'Surname': 'Bolton'}


In [16]:

sample_chars_query = """
FOR c IN Characters
    LIMIT 10
    RETURN c.name
"""

query_result = db.AQLQuery(sample_chars_query, rawResults=True)

for doc in  query_result:
    print(doc, sep='\n')

Ned
Robert
Jaime
Catelyn
Cersei
Daenerys
Jorah
Petyr
Viserys
Jon


In [17]:

sample_chars_query = """
FOR c IN Characters
    SORT c.name
    LIMIT 10
    RETURN { Name: c.name, Surname: c.surname, Age: c.age }
"""

query_result = db.AQLQuery(sample_chars_query, rawResults=True)

for doc in  query_result:
    print(doc, sep='\n')

{'Name': 'Arya', 'Surname': 'Stark', 'Age': 11}
{'Name': 'Bran', 'Surname': 'Stark', 'Age': 10}
{'Name': 'Brienne', 'Surname': 'Tarth', 'Age': 32}
{'Name': 'Bronn', 'Surname': None, 'Age': None}
{'Name': 'Catelyn', 'Surname': 'Stark', 'Age': 40}
{'Name': 'Cersei', 'Surname': 'Lannister', 'Age': 36}
{'Name': 'Daario', 'Surname': 'Naharis', 'Age': None}
{'Name': 'Daenerys', 'Surname': 'Targaryen', 'Age': 16}
{'Name': 'Davos', 'Surname': 'Seaworth', 'Age': 49}
{'Name': 'Ellaria', 'Surname': 'Sand', 'Age': None}


# 3. Tạo đồ thị

## Lấy data từ "shortest path data"

In [18]:
from pyArango.collection import Collection, Field, Edges
from pyArango.graph import Graph, EdgeDefinition

# Định nghĩa Collections
class Places(Collection):
    _fields = {
        "label": Field()
    }

class Connect(Edges):
    _fields = {
        "travelTime": Field()
    }

# Định nghĩa Graph
class TravelGraph(Graph):
    _edgeDefinitions = [
        EdgeDefinition("Connect", fromCollections=["Places"], toCollections=["Places"]),
        ]
    _orphanedCollections = []


if "Places" not in db.collections:
    db.createCollection(className="Places")

if "Connect" not in db.collections:
    db.createCollection(className="Connect")

# Tạo graph
if "TravelGraph" not in db.graphs:
    travel_graph = db.createGraph("TravelGraph")
else:
    travel_graph = db.graphs["TravelGraph"]


import json

# Đọc dữ liệu từ tệp JSON
with open("./shortest_path_data/places.json", "r") as f:
    places_data = json.load(f)
    db["Places"].importBulk(places_data)


with open("./shortest_path_data/connect.json", "r") as f:
    connections_data = json.load(f)
    db["Connect"].importBulk(connections_data)

print("Tạo đồ thị thành công")

Tạo đồ thị thành công


In [19]:
def get_graph_data():
    places_query = "FOR v IN Places RETURN {key: v._key, label: v.label}"
    connections_query = "FOR e IN Connect RETURN {from: e._from, to: e._to}"

    places = {doc["key"]: doc["label"] for doc in db.AQLQuery(places_query, rawResults=True)}
    connections = db.AQLQuery(connections_query, rawResults=True)

    # Xây dựng danh sách kề
    adjacency_list = {key: [] for key in places.keys()}
    for connection in connections:
        from_key = connection["from"].split("/")[-1]
        to_key = connection["to"].split("/")[-1]
        adjacency_list[from_key].append(to_key)
    return adjacency_list, places
adjacency_list, places = get_graph_data()
for key, value in adjacency_list.items():
    print(f"{key}: {value}")

Inverness: ['Aberdeen']
Aberdeen: ['Inverness', 'Leuchars']
Leuchars: ['Aberdeen', 'Edinburgh', 'StAndrews']
StAndrews: ['Leuchars']
Edinburgh: ['Leuchars', 'Glasgow', 'York']
Glasgow: ['Edinburgh', 'Carlisle']
York: ['Edinburgh', 'Carlisle', 'London']
Carlisle: ['Glasgow', 'York', 'Birmingham']
Birmingham: ['Carlisle', 'London']
London: ['Birmingham', 'York', 'Brussels']
Brussels: ['London', 'Cologne']
Cologne: ['Brussels']
Toronto: ['Winnipeg']
Winnipeg: ['Toronto', 'Saskatoon']
Saskatoon: ['Winnipeg', 'Edmonton']
Edmonton: ['Saskatoon', 'Jasper']
Jasper: ['Edmonton', 'Vancouver']
Vancouver: ['Jasper']


In [20]:
from GraphAlgo import *
import pandas as pd
start = 'Inverness'

free_ser        = pd.Series(1, index = adjacency_list.keys())
free_ser[start] = 0

seq = []
dfs(adjacency_list, free_ser, start, seq)
seq = ' --> '.join(seq)
print(f'Chuỗi thứ tự duyệt đồ thị từ đỉnh [{start}]:\n{seq}')

Chuỗi thứ tự duyệt đồ thị từ đỉnh [Inverness]:
Inverness --> Aberdeen --> Leuchars --> Edinburgh --> Glasgow --> Carlisle --> York --> London --> Birmingham --> Brussels --> Cologne --> StAndrews


In [21]:
components = dfs_all_components(adjacency_list)
for c in components:
    print(f'Thành phần liên thông #{components.index(c) + 1} = {c}')

Thành phần liên thông #1 = ['Inverness', 'Aberdeen', 'Leuchars', 'Edinburgh', 'Glasgow', 'Carlisle', 'York', 'London', 'Birmingham', 'Brussels', 'Cologne', 'StAndrews']
Thành phần liên thông #2 = ['Toronto', 'Winnipeg', 'Saskatoon', 'Edmonton', 'Jasper', 'Vancouver']


## Đường đi ngắn nhất

In [22]:
query = """
FOR p IN OUTBOUND K_SHORTEST_PATHS 'Places/Aberdeen' TO 'Places/London'
GRAPH 'TravelGraph'
    LIMIT 1
    RETURN {
        places: p.vertices[*].label,
        travelTimeTotal: SUM(p.edges[*].travelTime)
    }
"""

query_result = db.AQLQuery(query, rawResults=True)

for doc in query_result:
    print(doc, sep='\n')

{'places': ['Aberdeen', 'Leuchars', 'Edinburgh', 'York', 'London'], 'travelTimeTotal': 8.3}
